In [32]:
import pandas as pd
import numpy as np

In [151]:
train=pd.read_csv('train.csv',sep='|')
test=pd.read_csv('test.csv',sep='|')

Data Preparation: Add features to the data set

In [166]:
#train['totalScanTimeInMinutes'] = train['totalScanTimeInSeconds'] / 60.0
train['scannedLineItems'] = train['scannedLineItemsPerSecond'] * train['totalScanTimeInSeconds']
train['pricePerScannedLineItem'] = train['grandTotal'] / train['scannedLineItems']
train['scansWithoutRegistrationPerScannedLineItem'] = train['scansWithoutRegistration'] / train['scannedLineItems']
train['quantityModificationsPerScannedLineItem'] = train['quantityModifications'] / train['scannedLineItems']
train['lineItemVoidsPerSecond'] = train['lineItemVoids'] / train['totalScanTimeInSeconds']
#rain['scansWithoutRegistrationPerSecond'] = train['scansWithoutRegistration'] / train['totalScanTimeInSeconds']
#train['quantityModificationsPerSecond'] = train['quantityModifications'] / train['totalScanTimeInSeconds']
#train['secondsPerEuro'] = train['totalScanTimeInSeconds'] / train['grandTotal']
#train['lineItemVoidsPerEuro'] = train['lineItemVoids'] / train['grandTotal']
#train['scansWithoutRegistrationPerEuro'] = train['scansWithoutRegistration'] / train['grandTotal']
#train['quantityModificationsPerEuro'] = train['quantityModifications'] / train['grandTotal']

In [4]:
train.corr()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScanTimeInMinutes,scannedLineItems,pricePerScannedLineItem,scansWithoutRegistrationPerScannedLineItem,quantityModificationsPerScannedLineItem,lineItemVoidsPerSecond
trustLevel,1.000000,0.025264,0.047656,-0.017312,0.017296,0.006506,0.030701,0.027626,0.009241,-0.319765,0.025264,0.006357,0.008270,0.004769,0.001516,0.012482
totalScanTimeInSeconds,0.025264,1.000000,-0.001230,0.019068,0.009066,-0.011415,-0.262988,-0.208450,-0.011623,0.110414,1.000000,0.009769,-0.019434,-0.021238,-0.028598,-0.220757
grandTotal,0.047656,-0.001230,1.000000,0.006321,-0.030573,-0.008163,-0.011885,0.075679,-0.008858,0.001421,-0.001230,-0.006955,0.328316,0.003583,-0.004884,-0.005339
lineItemVoids,-0.017312,0.019068,0.006321,1.000000,-0.017023,-0.007173,-0.004438,0.005114,0.346514,0.063496,0.019068,0.021647,-0.025732,-0.014101,-0.016196,0.094364
scansWithoutRegistration,0.017296,0.009066,-0.030573,-0.017023,1.000000,0.018961,-0.024126,-0.018844,0.002834,0.074123,0.009066,-0.026529,0.002975,0.346238,0.003962,0.009702
quantityModifications,0.006506,-0.011415,-0.008163,-0.007173,0.018961,1.000000,-0.014880,0.010548,0.018318,-0.000864,-0.011415,-0.035571,0.014195,0.022659,0.368689,0.000753
scannedLineItemsPerSecond,0.030701,-0.262988,-0.011885,-0.004438,-0.024126,-0.014880,1.000000,0.692152,-0.056316,-0.023085,-0.262988,0.100724,-0.061587,-0.057469,-0.055892,0.653202
valuePerSecond,0.027626,-0.208450,0.075679,0.005114,-0.018844,0.010548,0.692152,1.000000,0.019886,-0.028873,-0.208450,-0.022396,0.033083,0.038250,0.040377,0.863587
lineItemVoidsPerPosition,0.009241,-0.011623,-0.008858,0.346514,0.002834,0.018318,-0.056316,0.019886,1.000000,-0.090116,-0.011623,-0.519169,0.610629,0.634480,0.616196,0.061248
fraud,-0.319765,0.110414,0.001421,0.063496,0.074123,-0.000864,-0.023085,-0.028873,-0.090116,1.000000,0.110414,0.298423,-0.100765,-0.090615,-0.094393,-0.026932


# Feature Selection

In [117]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

Y = train['fraud']
X = train.drop('fraud',axis=1)   

best_features = SelectKBest(f_classif, k=4).fit(X,Y)
best_feature_list = X.columns[best_features.get_support()]

X = X[best_feature_list]

X.columns

Index(['trustLevel', 'totalScanTimeInSeconds', 'scannedLineItems',
       'pricePerScannedLineItem'],
      dtype='object')

In [152]:
Y = train['fraud']
X = train.drop('fraud',axis=1)  

# Identify and handle multicollinearity

In [118]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    #return X.iloc[:, variables]

calculate_vif_(X)

Remaining variables:
Index(['trustLevel', 'totalScanTimeInSeconds', 'scannedLineItems',
       'pricePerScannedLineItem'],
      dtype='object')


# Statsmodel package

In [119]:
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split

x_train, x_holdout, y_train, y_holdout = train_test_split(X, Y, test_size=0.2, random_state=324)
    
    
model = sm.Logit(y_train, x_train)
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.062918
         Iterations 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fraud   No. Observations:                 1503
Model:                          Logit   Df Residuals:                     1499
Method:                           MLE   Df Model:                            3
Date:                Tue, 23 Apr 2019   Pseudo R-squ.:                  0.6946
Time:                        21:31:49   Log-Likelihood:                -94.566
converged:                       True   LL-Null:                       -309.60
                                        LLR p-value:                 6.772e-93
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
trustLevel                 -5.4807      0.669     -8.192      0.000      -6.792      -4.169
totalScanTimeInSeconds      0.0018      0.000      4.717      0.000       0.001       0.003
scannedLineItems            0.2256      0.028      7.938      0.000       0.170       0.281
pricePerScannedLineItem    -0.4903      0.134     -3.653      0.000      -0.753      -0.227
===========================================================================================

Possibly complete quasi-separation: A fraction 0.71 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [120]:
from sklearn.metrics import accuracy_score
predictions = round(result.predict(x_holdout))
accuracy_score(y_true = y_holdout, y_pred = predictions)

0.9680851063829787

# Scikit-learn Package

In [122]:
from sklearn.linear_model import LogisticRegression
np.set_printoptions(suppress=True)
 
x_train, x_holdout, y_train, y_holdout = train_test_split(X, Y, test_size=0.2, random_state=324)
    
    

model = LogisticRegression(C=1e80, solver='lbfgs', multi_class='multinomial', max_iter = 10000, fit_intercept = True)
model.fit(X, Y)

print(model.coef_, ", ", model.intercept_)

[[-10.081   0.004  -0.001   0.925   1.443   0.286  -7.535 -26.828  -6.524
    1.278   0.96  -13.863  -5.555  -4.677]] ,  [-31.047]


In [204]:
from sklearn.metrics import accuracy_score
predictions = model.predict(x_holdout)
accuracy_score(y_true = y_holdout, y_pred = predictions)

ValueError: X has 14 features per sample; expecting 12

# Cross-Validation (only possible with scikit-learn) [predict with cross_val_predict(model, X, y, cv=,,,)]

In [201]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

def my_custom_loss_func(y_true, y_pred):
    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    score = ((-25)*fp + (-5)*fn + 5*tp) / len(y_true)
    return (score)

my_custom_score = make_scorer(my_custom_loss_func, greater_is_better=True)
my_custom_loss_func(y_holdout, predictions)

skf = StratifiedKFold(n_splits=30)
loo = LeaveOneOut()

scores = cross_val_score(model, X, Y, cv=skf, scoring=my_custom_score)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.16 (+/- 0.39)


In [203]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_holdout,predictions)
cm

array([[350,   1],
       [  2,  23]], dtype=int64)

In [11]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_holdout,predictions)

0.9585754985754986

In [127]:
def get_monetary_value(cm, y_holdout):
        tn, fp, fn, tp = cm.ravel()
        print("True negative: ", tn)
        print("False positive: ", fp)
        print("False negative: ", fn)
        print("True positive: ", tp)
        score = (-25)*fp + (-5)*fn + 5*tp
        print(score, 'for ', len(y_holdout), ' instances in the test set')
        print(score/len(y_holdout), ' per instance in the test set')
        return score
        
monetary_value = get_monetary_value(cm, y_holdout)

True negative:  350
False positive:  1
False negative:  2
True positive:  23
80 for  376  instances in the test set
0.2127659574468085  per instance in the test set


# Best subset selection

In [219]:
import statsmodels.formula.api as sm
import itertools
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

disp=False

Y = train['fraud']
X = train.drop('fraud',axis=1)   

def my_custom_loss_func(y_true, y_pred):
    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    score = ((-25)*fp + (-5)*fn + 5*tp) / len(y_true)
    return (score)


def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = LogisticRegression(C=1e80, solver='lbfgs', multi_class='multinomial', max_iter = 10000, fit_intercept = True)
    regr = model.fit(X[list(feature_set)], Y)   
    monetary_value = my_custom_loss_func(Y, regr.predict(X[list(feature_set)]))
    feature_set = feature_set
    return {"model":regr, "MonetaryValue":monetary_value, "FeatureSet": feature_set}

def getBest(k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
        
        # Wrap everything up in a nice dataframe
        models = pd.DataFrame(results)
        
        # Choose the model with the highest RSS
        best_model = models.loc[models["MonetaryValue"].argmin()]
        
        print("Processed ", models.shape[0], "models on", k, "predictors")
        
        # Return the best model, along with some other useful information about the model
        return best_model
    
    
models = pd.DataFrame(columns=["MonetaryValue", "model", "FeatureSet"])

for i in range(1, len(list(X))):
    models.loc[i] = getBest(i)
    
#print(models.loc[models["MonetaryValue"].argmax()]["model"].summary()) 
    
print(models)    
    
model = models.loc[models["MonetaryValue"].argmax()]["model"]

feature_list = models.loc[models["MonetaryValue"].argmax()]["FeatureSet"]

print(feature_list)
    
# print(getBest(7)["model"].summary())

Processed  1 models on 1 predictors
Processed  1 models on 2 predictors
Processed  1 models on 3 predictors
Processed  1 models on 4 predictors
Processed  1 models on 5 predictors
Processed  1 models on 6 predictors
Processed  1 models on 7 predictors
Processed  1 models on 8 predictors
Processed  1 models on 9 predictors
Processed  1 models on 10 predictors
Processed  1 models on 11 predictors
Processed  1 models on 12 predictors
Processed  1 models on 13 predictors
    MonetaryValue                                              model  \
1       -0.276743  LogisticRegression(C=1e+80, class_weight=None,...   
2       -0.447046  LogisticRegression(C=1e+80, class_weight=None,...   
3       -0.383183  LogisticRegression(C=1e+80, class_weight=None,...   
4       -0.396487  LogisticRegression(C=1e+80, class_weight=None,...   
5       -0.463012  LogisticRegression(C=1e+80, class_weight=None,...   
6       -0.463012  LogisticRegression(C=1e+80, class_weight=None,...   
7       -0.463012  Logis